In [ ]:
#Can you return a table containing the top 10 stores by total transactions in the data?
#Make sure they’re sorted from highest to lowest.

transactions = pd.read_csv("../retail/transactions.csv", parse_dates=["date"])

(transactions
 .groupby(["store_nbr"])[["transactions"]] 
 .sum() 
 .sort_values("transactions", ascending=False)  # sort in descending order
 .iloc[:10]  
)

In [ ]:
#Can you get me the total transactions by store and month?
#Sort the table from first month to last, then by highest transactions to lowest within each month

transactions["month"] = transactions["date"].dt.month
(transactions
 .groupby(["store_nbr", "month"])[["transactions"]]
 .sum()
 .sort_values(["month", "transactions"], ascending=[True, False] # ascending month, descending transactions
))

In [ ]:
#Can you help me access rows and columns with multiple indices? I’ve been struggling with multi-index DataFrames.
#Access:
#Grab Store 3, Month 1
#Then, select the column storing the mean of transactions
#Fix:
#Drop the outer layer of the column Index
#Reset the row index so it is the default integer index

grouped = (
    transactions.groupby(["store_nbr", "month"])
    .agg({"transactions": ["sum", "mean"]})
    .sort_values(by=["month", ("transactions", "sum")], ascending=[True, False])
)

grouped.loc[(3, 1)]


# Grab mean column in column multi-index

grouped.loc[:, [("transactions", "mean")]]

# Drop level from column index (axis=1), then reset index

grouped.droplevel(0, axis=1).reset_index()

In [ ]:
#Calculate the mean of target met by store, and the sum of bonuses to be paid to each store.
#Sort them by highest to lowest bonus payout.
#Then, do the same for day of week and month.

# Recreate table from section 3

transactions = transactions.assign(
    target_pct=transactions["transactions"] / 2500,
    met_target=(transactions["transactions"] / 2500) >= 1,
    bonus_payable=((transactions["transactions"] / 2500) >= 1) * 100,
    month=transactions.date.dt.month,
    day_of_week=transactions.date.dt.dayofweek,
)

# Group transactions by store number to get store level stats 
# use agg to calculate mean of met_target, sum of transactions by store
# sort values by bonus payable in descending order

(transactions.groupby("store_nbr")
 .agg({"met_target": "mean", "bonus_payable": "sum"})
 .sort_values(by=["bonus_payable"], ascending=False)
)

In [ ]:
#Calculate the mean of transactions by store number and day of week while keeping row numbers.
#Then compare the performance of each row to its day of week average. (difference between transactions and daily avg)

# Assign method for column creation

transactions.assign(
    # use transform to create store level average transactions by day
    avg_store_transactions = (transactions
                              .groupby(["store_nbr", "day_of_week"])
                              ["transactions"]
                              .transform("mean")),
    
    # Then create a difference column subtracting the store average from that days transactions
    difference = lambda x: x["transactions"] - x["avg_store_transactions"]
)